In [27]:
import jovian

In [28]:
# Execute this to save new versions of the notebook
jovian.commit(project="yahoo-finance-web-scraper")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[jovian] Error: Failed to read the Jupyter notebook. Please re-run this cell to try again. If the issue persists, provide the "filename" argument to "jovian.commit" e.g. "jovian.commit(filename='my-notebook.ipynb')"


In [29]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [30]:
my_url = 'https://finance.yahoo.com/topic/stock-market-news/'

headers = {"User-Agent": "Mozilla/5.0"}
cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}

response = requests.get(my_url, headers=headers)

In [31]:
print("response.ok : {} , response.status_code : {}".format(response.ok , response.status_code))

response.ok : True , response.status_code : 200


In [32]:
print("Preview of response.text : ", response.text[:500])

Preview of response.text :  <!DOCTYPE html><html data-color-theme="light" id="atomic" class="NoJs desktop" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charSet="utf-8"/><title>Latest Stock Market News</title><meta name="keywords" content="401k, Business, Financial Information, Investing, Investor, Market News, Stock Research, Stock Valuation, business news, economy, finance, investment tools, mortgage, mutual 


In [33]:
def get_page(url):
    """Download a webpage and return a beautiful soup doc"""
    response = requests.get(url, headers=headers)
    if not response.ok:
        print('Status code:', response.status_code)
        raise Exception('Failed to load page {}'.format(url))
    page_content = response.text
    doc = BeautifulSoup(page_content, 'html.parser')
    return doc

In [34]:
doc = get_page(my_url)
print('Type of doc: ',type(doc))

Type of doc:  <class 'bs4.BeautifulSoup'>


In [35]:
doc.find('title')

<title>Latest Stock Market News</title>

In [36]:
div_tags = doc.find_all('div', {'class': "Ov(h) Pend(44px) Pstart(25px)"})

In [37]:
len(div_tags)

2

In [38]:
print(div_tags[1])

<div class="Ov(h) Pend(44px) Pstart(25px)"><div class="Fz(12px) Fw(b) Tt(c) D(ib) Mb(6px) C($c-fuji-blue-1-a) Mend(9px) Mt(-2px)" data-test-locator="catlabel">Business</div><div class="C(#959595) Fz(11px) D(ib) Mb(6px)">Yahoo Finance</div><h3 class="Mb(5px)"><a class="js-content-viewer wafer-caas Fw(b) Fz(18px) Lh(23px) LineClamp(2,46px) Fz(17px)--sm1024 Lh(19px)--sm1024 LineClamp(2,38px)--sm1024 mega-item-header-link Td(n) C(#0078ff):h C(#000) LineClamp(2,46px) LineClamp(2,38px)--sm1024 not-isInStreamVideoEnabled" data-uuid="37eaa63f-bd7f-4304-ad0e-88bb03c1eddd" data-wf-caas-prefetch="1" data-wf-caas-uuid="37eaa63f-bd7f-4304-ad0e-88bb03c1eddd" href="/news/carvana-stock-tanks-december-7-154119414.html"><u class="StretchedBox"></u>Carvana stock collapses amid bankruptcy fears after creditor pact and another $1 price target</a></h3><p class="Fz(14px) Lh(19px) Fz(13px)--sm1024 Lh(17px)--sm1024 LineClamp(2,38px) LineClamp(2,34px)--sm1024 M(0)">Carvana shares spiraled Wednesday after the on

In [39]:
print("Source: ", div_tags[1].find('div').text)
print("Head Line : {}".format(div_tags[1].find('a').text))

Source:  Business
Head Line : Carvana stock collapses amid bankruptcy fears after creditor pact and another $1 price target


In [40]:
def get_news_tags(doc):
    """Get the list of tags containing news information"""
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list

In [41]:
BASE_URL = 'https://finance.yahoo.com' #Global Variable 

def parse_news(news_tag):
    """Get the news data point and return dictionary"""
    news_source = news_tag.find('div').text #source
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
            'image' : news_image
           }

In [42]:
import pandas as pd

In [43]:
def scrape_yahoo_news(url, path=None):
    """Get the yahoo finance market news and write them to CSV file """
    if path is None:
        path = 'stock-market-news.csv'
        
    print('Requesting html page')
    doc = get_page(url)

    print('Extracting news tags')
    news_list = get_news_tags(doc)

    print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]

    print('Save the data to a CSV')
    news_df = pd.DataFrame(news_data)
    news_df.to_csv(path, index=None)
    
    #This return statement is optional, we are doing this just analyze the final output 
    return news_df 

In [44]:
YAHOO_NEWS_URL = BASE_URL+'/topic/stock-market-news/'
news_df = scrape_yahoo_news(YAHOO_NEWS_URL)

Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV


In [45]:
news_df.head()

,source,headline,url,content,image
0,Business,"Stock Market Closes Lower, Extending Losses As...",https://finance.yahoo.com/m/e7e38cf2-4454-3cff...,The stock market traded lower as corporate ear...,https://s.yimg.com/uu/api/res/1.2/i8ZrbsVjvTkq...
1,Business,"Casey's Earnings Jump, Ollie's Bargain Outlet ...",https://finance.yahoo.com/m/73e51005-676b-3369...,Casey's was mixed on estimates while Ollie's B...,https://s.yimg.com/uu/api/res/1.2/q.GeVpjYebkC...
